# Looking at measurement coverage patterns

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.display.max_columns = None

In [12]:
%%time
result = pd.read_csv('../data/water/CA-result-withmeasuregroup.csv', low_memory=False)
result = result.ix[result.StartDate >= '2010-01-01']
station = pd.read_csv('../data/water/CA-station-clean.csv', low_memory=False)
data = pd.merge(result, station, left_on='LocationIdentifier', right_on='MonitoringLocationId')

data.StartDate = pd.to_datetime(data.StartDate)
data['Year'] = data.StartDate.dt.year

del result
del station

CPU times: user 472 ms, sys: 82.5 ms, total: 555 ms
Wall time: 577 ms


In [13]:
data.columns

Index(['LocationIdentifier', 'Medium', 'MediumSubdivision', 'StartDate',
       'StartTime', 'TimeZone', 'Category', 'Pollutant', 'Unit', 'Mclg', 'Mcl',
       'Value', 'ExceedsMclg', 'ExceedsMcl', 'Comment', 'WarningLevel',
       'Organization', 'MonitoringLocationId', 'MonitoringLocationName',
       'MonitoringLocationType', 'MonitoringLocationDescription', 'HUC',
       'DrainageArea', 'DrainageAreaUnit', 'ContributingDrainageArea',
       'ContributingDrainageAreaUnit', 'Latitude', 'Longitude',
       'VerticalMeasure', 'VerticalMeasureUnit', 'StateCode', 'CountyCode',
       'CountyName', 'AquiferName', 'FormationType', 'AquiferType', 'Provider',
       'Edits', 'Year'],
      dtype='object')

## Getting annual per-county mean values to use for zero filling

The idea is, when there are NA values at specific sites, we can zero fill them by putting in the averages for that county.

In [27]:
values = data[['CountyName', 'Year', 'Pollutant', 'Value']]
countyMeans = values.groupby(['CountyName', 'Year', 'Pollutant'])\
                    .mean()\
                    .unstack()
print("Length: {}".format(len(countyMeans)))
countyMeans.describe()

Length: 350


Value                                                  \
Pollutant    Antimony     Arsenic      Barium   Beryllium     Cadmium   
count      167.000000  301.000000  200.000000  139.000000  257.000000   
mean         0.000140    0.007462    0.073205    0.000057    0.000064   
std          0.000306    0.051854    0.089056    0.000273    0.000237   
min          0.000000    0.000090    0.000000    0.000000   -0.000100   
25%          0.000002    0.001500    0.028758    0.000000    0.000000   
50%          0.000055    0.002534    0.047341    0.000002    0.000013   
75%          0.000134    0.004179    0.088400    0.000020    0.000057   
max          0.002091    0.893500    0.719000    0.002400    0.002883   

                                                                      \
Pollutant    Chromium      Copper    Fluoride       HAA5        Lead   
count      263.000000  301.000000  181.000000  27.000000  271.000000   
mean         0.003444    0.004005    0.347093   0.116912    0.001109   
std          0.017085    0.007333    0.405766   0.043379    0.004266   
min          0.000000    0.000250    0.000000   0.059500    0.000000   
25%          0.000445    0.001194    0.104615   0.086205    0.000048   
50%          0.000902    0.002073    0.231250   0.105213    0.000190   
75%          0.002013    0.003939    0.433400   0.150736    0.000509   
max          0.221879    0.079450    2.757628   0.212892    0.042200   

                                                                           \
Pollutant       Mercury     Nitrate     Nitrite          PCBs    Selenium   
count      2.090000e+02  304.000000  196.000000  5.300000e+01  282.000000   
mean       1.138397e-05    5.959260    0.093966 -3.694380e-06    0.001869   
std        3.489223e-05   10.640079    0.201315  3.173399e-04    0.004813   
min        0.000000e+00    0.000000   -0.006500 -2.104500e-03    0.000000   
25%        3.373333e-07    0.491375    0.005098  0.000000e+00    0.000105   
50%        1.150000e-06    2.069624    0.017945  0.000000e+00    0.000359   
75%        5.210926e-06    6.939570    0.089060  1.903000e-09    0.001309   
max        3.530000e-04   80.950000    1.400000  6.467700e-04    0.038229   

                                              
Pollutant    Simazine       TTHMs     Xylene  
count      113.000000  127.000000  60.000000  
mean         0.000205    0.043635   0.000035  
std          0.001379    0.101061   0.000087  
min          0.000000    0.000000   0.000000  
25%          0.000007    0.000010   0.000000  
50%          0.000017    0.000197   0.000000  
75%          0.000037    0.001549   0.000020  
max          0.014497    0.474391   0.000520

In [28]:
countyMeans

Value                                              \
Pollutant              Antimony   Arsenic    Barium Beryllium       Cadmium   
CountyName       Year                                                         
Alameda County   2010  0.000000  0.001852  0.029556  0.000000  8.950000e-06   
                 2011  0.000000  0.001542  0.024880  0.000000  1.895161e-05   
                 2012  0.000000  0.002000  0.042333  0.000000  0.000000e+00   
                 2013  0.000000  0.002539  0.037636  0.000000  4.132653e-05   
                 2014  0.000000  0.003704  0.048311  0.000000  9.306122e-06   
                 2015  0.000011  0.002960  0.051480  0.000000  0.000000e+00   
                 2016       NaN       NaN       NaN       NaN           NaN   
Alpine County    2010       NaN       NaN       NaN       NaN           NaN   
                 2012       NaN       NaN       NaN       NaN           NaN   
                 2013       NaN       NaN       NaN       NaN           NaN   
                 2014       NaN       NaN       NaN       NaN           NaN   
Amador County    2011  0.000000  0.002143  0.104643  0.000000  0.000000e+00   
                 2012  0.000000  0.002357  0.138429  0.000000  0.000000e+00   
Butte County     2010  0.000029  0.000791  0.007563  0.000000  1.538462e-07   
                 2011  0.000000  0.000456  0.000541  0.000000  0.000000e+00   
                 2012  0.000000  0.000487  0.009357  0.000000  0.000000e+00   
                 2013  0.000000  0.000441  0.013450  0.000000  0.000000e+00   
                 2014  0.000000  0.000617  0.012375  0.000000  0.000000e+00   
                 2015  0.000000  0.000435  0.013097  0.000000  0.000000e+00   
Calaveras County 2010       NaN  0.006217  0.025033       NaN           NaN   
                 2012  0.000000  0.006778  0.076333  0.000000  0.000000e+00   
                 2013       NaN       NaN       NaN       NaN           NaN   
                 2014       NaN       NaN       NaN       NaN           NaN   
                 2015       NaN       NaN       NaN       NaN           NaN   
                 2016       NaN       NaN       NaN       NaN           NaN   
Colusa County    2010  0.000093  0.004432  0.053350       NaN  4.632653e-06   
                 2011       NaN  0.010422  0.072450       NaN  0.000000e+00   
                 2012       NaN  0.027035  0.014600  0.000054  1.127273e-05   
                 2013       NaN  0.005690       NaN       NaN  0.000000e+00   
                 2014       NaN  0.006620       NaN       NaN  0.000000e+00   
...                         ...       ...       ...       ...           ...   
Trinity County   2014       NaN  0.000473       NaN       NaN  0.000000e+00   
                 2015       NaN  0.000490       NaN       NaN  0.000000e+00   
Tulare County    2010  0.000063  0.000665  0.054150  0.000075  1.100000e-05   
                 2011       NaN       NaN       NaN       NaN           NaN   
                 2012  0.000063  0.000580  0.073733       NaN           NaN   
                 2013  0.000068  0.003200  0.039900       NaN           NaN   
                 2014  0.000066  0.001093  0.108037       NaN  1.018571e-04   
                 2015  0.000082  0.009356  0.100340  0.000037  1.338500e-04   
                 2016       NaN       NaN       NaN       NaN           NaN   
Tuolumne County  2013       NaN       NaN       NaN       NaN           NaN   
                 2014       NaN       NaN       NaN       NaN           NaN   
Ventura County   2010       NaN  0.000980       NaN       NaN  1.250000e-04   
                 2011  0.000089  0.001075  0.046678  0.000016  1.171000e-04   
                 2012       NaN  0.000989       NaN       NaN  2.857143e-05   
                 2013       NaN  0.004562       NaN       NaN  5.777778e-05   
                 2014       NaN  0.007313       NaN       NaN  1.427273e-04   
                 2015       NaN       NaN       NaN       NaN           NaN 

There are still quite a lot of NAs - 349 rows, and none of them are full for all the pollutants.

Will need to think if there's a way to fill that in. Based on the pattern of missing values, I'm not sure interpolation is appropriate, because frequently it's not that in-between values are missing. Forward-filling and back-filling might work. If values are NaN all the way through, maybe we just take the average from all neighboring counties.  For now, though, I'm thinking filling with the state-wide averages for that year will do for a first pass.

In [48]:
stateMeans = values[['Year', 'Pollutant', 'Value']]\
                    .groupby(['Year', 'Pollutant'])\
                    .mean()\
                    .unstack()
print("Length: {}".format(len(stateMeans)))
stateMeans.describe()

Length: 7


Value                                                    \
Pollutant  Antimony   Arsenic    Barium Beryllium   Cadmium  Chromium   
count      7.000000  7.000000  7.000000  7.000000  7.000000  7.000000   
mean       0.000163  0.007312  0.060953  0.000032  0.000075  0.004533   
std        0.000140  0.003829  0.028510  0.000045  0.000106  0.004437   
min        0.000039  0.003716  0.040964  0.000005  0.000014  0.001210   
25%        0.000063  0.004707  0.044343  0.000007  0.000028  0.001422   
50%        0.000087  0.005467  0.053983  0.000015  0.000036  0.002514   
75%        0.000251  0.009097  0.059967  0.000029  0.000054  0.006212   
max        0.000387  0.014395  0.123107  0.000132  0.000314  0.012736   

                                                                       \
Pollutant    Copper  Fluoride      HAA5      Lead   Mercury   Nitrate   
count      7.000000  7.000000  6.000000  7.000000  7.000000  7.000000   
mean       0.003587  0.764661  0.121194  0.000976  0.000025  7.687445   
std        0.000728  0.197230  0.030517  0.000892  0.000029  1.428797   
min        0.002686  0.535266  0.074044  0.000135  0.000002  5.805821   
25%        0.003102  0.616235  0.106292  0.000489  0.000007  6.633025   
50%        0.003383  0.685155  0.122375  0.000558  0.000013  7.954041   
75%        0.004072  0.950588  0.143458  0.001182  0.000030  8.514417   
max        0.004695  0.998562  0.156952  0.002794  0.000083  9.757366   

                                                                           
Pollutant   Nitrite          PCBs  Selenium  Simazine     TTHMs    Xylene  
count      7.000000  6.000000e+00  7.000000  7.000000  6.000000  6.000000  
mean       0.152921  1.813945e-05  0.004223  0.000109  0.111534  0.000019  
std        0.086498  5.321911e-05  0.004079  0.000096  0.033307  0.000020  
min        0.034022 -6.516647e-05  0.000470  0.000017  0.059553  0.000000  
25%        0.102136  3.125000e-08  0.000709  0.000049  0.099668  0.000002  
50%        0.158898  1.373491e-05  0.003986  0.000103  0.108590  0.000012  
75%        0.187245  5.111121e-05  0.006051  0.000119  0.136684  0.000036  
max        0.298761  8.750000e-05  0.011586  0.000308  0.149731  0.000045

In [51]:
stateMeans

Value                                                    \
Pollutant  Antimony   Arsenic    Barium Beryllium   Cadmium  Chromium   
Year                                                                    
2010       0.000087  0.004634  0.040964  0.000015  0.000036  0.002514   
2011       0.000387  0.008232  0.043418  0.000009  0.000024  0.008527   
2012       0.000325  0.009962  0.053983  0.000005  0.000031  0.012736   
2013       0.000079  0.003716  0.045267  0.000033  0.000040  0.001210   
2014       0.000047  0.005467  0.057711  0.000025  0.000067  0.001503   
2015       0.000039  0.004781  0.062223  0.000006  0.000014  0.001341   
2016       0.000177  0.014395  0.123107  0.000132  0.000314  0.003897   

                                                                       \
Pollutant    Copper  Fluoride      HAA5      Lead   Mercury   Nitrate   
Year                                                                    
2010       0.002832  0.535266  0.147462  0.000558  0.000039  7.954688   
2011       0.003383  0.685155  0.156952  0.000501  0.000013  9.074146   
2012       0.004146  0.656946  0.103955  0.001307  0.000020  7.954041   
2013       0.003998  0.910391  0.074044  0.000477  0.000008  9.757366   
2014       0.004695  0.998562  0.113303  0.001057  0.000006  5.805821   
2015       0.002686  0.575524  0.131447  0.000135  0.000002  6.892500   
2016       0.003372  0.990785       NaN  0.002794  0.000083  6.373551   

                                                                           
Pollutant   Nitrite          PCBs  Selenium  Simazine     TTHMs    Xylene  
Year                                                                       
2010       0.078402 -6.516647e-05  0.011586  0.000103  0.097179  0.000042  
2011       0.207345  2.734482e-05  0.006273  0.000308  0.149731  0.000045  
2012       0.158898  0.000000e+00  0.005828  0.000044  0.107136  0.000001  
2013       0.298761  5.903333e-05  0.003986  0.000104  0.059553  0.000006  
2014       0.167146  1.250000e-07  0.000752  0.000133  0.145564  0.000018  
2015       0.125869  8.750000e-05  0.000470  0.000017  0.110043  0.000000  
2016       0.034022           NaN  0.000666  0.000055       NaN       NaN

In [100]:
# goign to just fillna the values for 2016

stateMeans.fillna(method='pad', inplace=True)
stateMeans

Value                                                    \
Pollutant  Antimony   Arsenic    Barium Beryllium   Cadmium  Chromium   
Year                                                                    
2010       0.000087  0.004634  0.040964  0.000015  0.000036  0.002514   
2011       0.000387  0.008232  0.043418  0.000009  0.000024  0.008527   
2012       0.000325  0.009962  0.053983  0.000005  0.000031  0.012736   
2013       0.000079  0.003716  0.045267  0.000033  0.000040  0.001210   
2014       0.000047  0.005467  0.057711  0.000025  0.000067  0.001503   
2015       0.000039  0.004781  0.062223  0.000006  0.000014  0.001341   
2016       0.000177  0.014395  0.123107  0.000132  0.000314  0.003897   

                                                                       \
Pollutant    Copper  Fluoride      HAA5      Lead   Mercury   Nitrate   
Year                                                                    
2010       0.002832  0.535266  0.147462  0.000558  0.000039  7.954688   
2011       0.003383  0.685155  0.156952  0.000501  0.000013  9.074146   
2012       0.004146  0.656946  0.103955  0.001307  0.000020  7.954041   
2013       0.003998  0.910391  0.074044  0.000477  0.000008  9.757366   
2014       0.004695  0.998562  0.113303  0.001057  0.000006  5.805821   
2015       0.002686  0.575524  0.131447  0.000135  0.000002  6.892500   
2016       0.003372  0.990785  0.131447  0.002794  0.000083  6.373551   

                                                                           
Pollutant   Nitrite          PCBs  Selenium  Simazine     TTHMs    Xylene  
Year                                                                       
2010       0.078402 -6.516647e-05  0.011586  0.000103  0.097179  0.000042  
2011       0.207345  2.734482e-05  0.006273  0.000308  0.149731  0.000045  
2012       0.158898  0.000000e+00  0.005828  0.000044  0.107136  0.000001  
2013       0.298761  5.903333e-05  0.003986  0.000104  0.059553  0.000006  
2014       0.167146  1.250000e-07  0.000752  0.000133  0.145564  0.000018  
2015       0.125869  8.750000e-05  0.000470  0.000017  0.110043  0.000000  
2016       0.034022  8.750000e-05  0.000666  0.000055  0.110043  0.000000

## Padding county-level data

In [102]:
%%time
def fillCounty(df, county):
    countydf = df.ix[county]\
                 .fillna(method='pad')\
                 .fillna(method='backfill')\
                 .reset_index()
    countydf['CountyName'] = county
    return countydf

filled = pd.concat([fillCounty(countyMeans, county) 
                    for county 
                    in data.CountyName.unique()])

CPU times: user 216 ms, sys: 6.72 ms, total: 223 ms
Wall time: 224 ms


In [103]:
filled

Year     Value                                              \
Pollutant        Antimony   Arsenic    Barium     Beryllium   Cadmium   
0          2010  0.000000  0.003728  0.023000  0.000000e+00  0.000000   
1          2011  0.000000  0.001693  0.031931  0.000000e+00  0.000018   
2          2012  0.000007  0.002653  0.045909  2.727273e-07  0.000031   
3          2013  0.000000  0.002404  0.041813  0.000000e+00  0.000005   
4          2014  0.000000  0.003250  0.036292  0.000000e+00  0.000000   
5          2015  0.000000  0.003474  0.044000  0.000000e+00  0.000000   
6          2016  0.000000  0.003474  0.044000  0.000000e+00  0.000000   
0          2010  0.000100  0.004190  0.020332           NaN  0.000000   
1          2011  0.000084  0.005375  0.030153           NaN  0.000000   
2          2012  0.000084  0.005301  0.030153           NaN  0.000000   
3          2013  0.000084  0.008310  0.030153           NaN  0.000000   
4          2014  0.000084  0.016710  0.030153           NaN  0.000000   
5          2015  0.000084  0.014192  0.030153           NaN  0.000000   
0          2010  0.000000  0.003312  0.000000  0.000000e+00  0.000063   
1          2011  0.000000  0.001998  0.002833  0.000000e+00  0.000028   
2          2012  0.000000  0.002653  0.029833  0.000000e+00  0.000015   
3          2013  0.000000  0.002323  0.030083  0.000000e+00  0.000011   
4          2014  0.000000  0.003257  0.031521  0.000000e+00  0.000000   
5          2015  0.000000  0.002869  0.038263  0.000000e+00  0.000000   
6          2016  0.000000  0.002167  0.038263  0.000000e+00  0.000000   
0          2010  0.000028  0.029077  0.004647  6.263158e-06  0.000102   
1          2011  0.000000  0.033242  0.000660  0.000000e+00  0.000000   
2          2012  0.000000  0.024938  0.021552  0.000000e+00  0.000000   
3          2013  0.000000  0.002375  0.031750  0.000000e+00  0.000000   
4          2014  0.000003  0.003312  0.034693  0.000000e+00  0.000000   
5          2015  0.000014  0.012488  0.048323  1.937500e-05  0.000057   
0          2010  0.000180  0.006315  0.031996  9.926471e-05  0.000024   
1          2011  0.000404  0.008930  0.028030  6.722222e-06  0.000013   
2          2012  0.000156  0.010947  0.027272  3.136842e-05  0.000041   
3          2013  0.000169  0.008590  0.031899  7.297297e-06  0.000076   
..          ...       ...       ...       ...           ...       ...   
3          2013  0.000017  0.002252  0.125644  7.777778e-07  0.000000   
4          2014  0.000017  0.003323  0.125644  7.777778e-07  0.000000   
5          2015  0.000017  0.003743  0.125644  7.777778e-07  0.000000   
6          2016  0.000017  0.003743  0.125644  7.777778e-07  0.000000   
0          2010  0.000093  0.004432  0.053350  5.400000e-05  0.000005   
1          2011  0.000093  0.010422  0.072450  5.400000e-05  0.000000   
2          2012  0.000093  0.027035  0.014600  5.400000e-05  0.000011   
3          2013  0.000093  0.005690  0.014600  5.400000e-05  0.000000   
4          2014  0.000093  0.006620  0.014600  5.400000e-05  0.000000   
5          2015  0.000093  0.002375  0.014600  5.400000e-05  0.000000   
0          2010       NaN  0.009800       NaN           NaN  0.000180   
1          2011       NaN  0.004017       NaN           NaN  0.000070   
2          2013       NaN  0.000730       NaN           NaN  0.000070   
3          2014       NaN  0.003617       NaN           NaN  0.000187   
4          2015       NaN  0.000920       NaN           NaN  0.000187   
5          2016       NaN  0.000920       NaN           NaN  0.000187   
0          2010       NaN  0.001905       NaN           NaN  0.000096   
1          2011       NaN  0.001905       NaN           NaN  0.000096   
2          2012       NaN  0.001905       NaN           NaN  0.000096   
3          2013       NaN  0.001905       NaN           NaN  0.000096   
4          2014       NaN  0.001905       NaN           NaN  0.000096   
5          2015       NaN  0.001905       NaN           NaN  0.000096

## Filling statewide annual averages for missing county data.

In [111]:
def fillnas(year):
    yeardf = filled.loc[filled.Year == year]
    yeardefaults = stateMeans.ix[year]
    return yeardf.fillna(yeardefaults)

filled = pd.concat([fillnas(year) for year in filled.Year.unique()])

filled

Year     Value                                                  \
Pollutant        Antimony   Arsenic    Barium     Beryllium       Cadmium   
0          2010  0.000000  0.003728  0.023000  0.000000e+00  0.000000e+00   
0          2010  0.000100  0.004190  0.020332  1.460182e-05  0.000000e+00   
0          2010  0.000000  0.003312  0.000000  0.000000e+00  6.290000e-05   
0          2010  0.000028  0.029077  0.004647  6.263158e-06  1.015714e-04   
0          2010  0.000180  0.006315  0.031996  9.926471e-05  2.430435e-05   
0          2010  0.000093  0.000805  0.009900  2.400000e-05  1.002000e-04   
0          2010  0.000061  0.002619  0.094059  1.900000e-05  3.320000e-04   
0          2010  0.000087  0.001796  0.040964  1.460182e-05  0.000000e+00   
0          2010  0.000050  0.006779  0.018179  2.880000e-05  1.473913e-04   
0          2010  0.000327  0.004915  0.062136  8.000000e-06  9.570000e-05   
0          2010  0.002000  0.002864  0.098703  2.712500e-05  2.883333e-03   
0          2010  0.000176  0.001099  0.035117  0.000000e+00  2.666667e-05   
0          2010  0.000322  0.004004  0.057391  1.916667e-06  1.084706e-04   
0          2010  0.000023  0.005046  0.189092  0.000000e+00  5.600000e-06   
0          2010  0.000047  0.000852  0.115550  1.460182e-05  2.142857e-06   
0          2010  0.000129  0.002900  0.000000  1.383333e-05  1.155000e-04   
0          2010  0.000087  0.002308  0.040964  1.460182e-05  3.617639e-05   
0          2010  0.000035  0.000527  0.032133  1.100000e-05  1.800000e-05   
0          2010  0.000089  0.000980  0.046678  1.616667e-05  1.250000e-04   
0          2010  0.000087  0.004634  0.040964  1.460182e-05  3.617639e-05   
0          2010  0.000099  0.014104  0.049968  1.714286e-05  2.435714e-05   
0          2010  0.000134  0.001639  0.076992  7.000000e-06  3.500000e-05   
0          2010  0.000157  0.002157  0.017641  1.527273e-05  1.151500e-04   
0          2010  0.000089  0.006314  0.110905  3.450000e-05  3.660000e-05   
0          2010  0.000087  0.003241  0.067300  7.000000e-06  1.036250e-04   
0          2010  0.001082  0.003976  0.014835  2.266667e-05  3.610000e-04   
0          2010  0.000042  0.001134  0.043444  5.000000e-07  6.000000e-06   
0          2010  0.000087  0.004634  0.040964  1.460182e-05  3.617639e-05   
0          2010  0.000036  0.003426  0.030548  1.200000e-05  2.250000e-06   
0          2010  0.000000  0.002000  0.011600  0.000000e+00  0.000000e+00   
..          ...       ...       ...       ...           ...           ...   
5          2015  0.000039  0.001905  0.062223  6.105405e-06  9.645000e-05   
5          2015  0.000000  0.000495  0.072857  0.000000e+00  0.000000e+00   
6          2016  0.000000  0.003474  0.044000  0.000000e+00  0.000000e+00   
6          2016  0.000000  0.002167  0.038263  0.000000e+00  0.000000e+00   
6          2016  0.000173  0.012149  0.036053  1.771250e-04  2.896923e-04   
6          2016  0.000075  0.002000  0.131740  2.595000e-04  7.826761e-05   
6          2016  0.000134  0.010900  0.065250  5.581765e-04  2.060000e-03   
6          2016  0.000177  0.002008  0.123107  1.322857e-04  0.000000e+00   
6          2016  0.000333  0.002600  0.148500  2.550000e-05  3.300000e-05   
6          2016  0.000134  0.000822  0.266875  2.712500e-05  8.166667e-05   
6          2016  0.000048  0.001752  0.156200  2.500000e-05  3.500000e-05   
6          2016  0.000073  0.010907  0.047857  0.000000e+00  0.000000e+00   
6          2016  0.000055  0.001350  0.453000  1.100000e-05  1.590000e-05   
6          2016  0.000034  0.061000  0.145200  1.322857e-04  0.000000e+00   
6          2016  0.000177  0.002000  0.123107  1.322857e-04  3.138065e-04   
6          2016  0.000099  0.001967  0.008015  1.714286e-05  1.200000e-04   
6          2016  0.000274  0.008800  0.066667  3.090000e-04  4.533333e-05   
6          2016  0.000049  0.002178  0.066033  6.100000e-05  3.400000e-05   
6          2016  0.000033  0.000927  0.054500  6.000000e-06  1.748000e-05   
6      

# Grouping by day

In [192]:
%%time
values = data[['CountyName', 'LocationIdentifier', 'Year', 'StartDate', 'Pollutant', 'Value']]
dailies = values.groupby(['CountyName', 'LocationIdentifier', 'Year', 'StartDate', 'Pollutant'])\
                .mean()\
                .unstack()

CPU times: user 115 ms, sys: 24.1 ms, total: 139 ms
Wall time: 137 ms


In [193]:
dailies

Value            \
Pollutant                                             Antimony   Arsenic   
CountyName     LocationIdentifier    Year StartDate                        
Alameda County CALWR_WQX-B9C74701355 2010 2010-03-17       NaN       NaN   
                                          2010-04-14       NaN       NaN   
                                          2010-06-23       NaN       NaN   
                                          2010-07-21       NaN       NaN   
                                          2010-08-30       NaN       NaN   
                                          2010-09-15       NaN       NaN   
                                          2010-10-13       NaN       NaN   
                                          2010-10-26       NaN       NaN   
                                          2010-11-03       NaN       NaN   
                                          2010-11-09       NaN       NaN   
                                          2010-12-07       NaN       NaN   
                                          2010-12-08       NaN       NaN   
                                     2011 2011-01-04       NaN       NaN   
                                          2011-01-05       NaN       NaN   
                                          2011-01-19       NaN       NaN   
                                          2011-02-02       NaN       NaN   
                                          2011-02-08       NaN       NaN   
                                          2011-03-02       NaN       NaN   
                                          2011-03-08       NaN       NaN   
                                          2011-03-23       NaN       NaN   
                                          2011-04-05       NaN       NaN   
                                          2011-04-07       NaN       NaN   
                                          2011-05-03       NaN       NaN   
                                          2011-05-04       NaN       NaN   
                                          2011-06-01       NaN       NaN   
                                          2011-06-07       NaN       NaN   
                                          2011-06-15       NaN       NaN   
                                          2011-07-06       NaN       NaN   
                                          2011-07-12       NaN       NaN   
                                          2011-08-03       NaN       NaN   
...                                                        ...       ...   
Yuba County    CALWR_WQX-A0052500    2010 2010-01-26       NaN  0.001228   
                                          2010-03-02       NaN  0.001670   
                                          2010-03-24       NaN  0.001630   
                                          2010-04-21       NaN  0.002305   
                                          2010-05-26       NaN  0.001385   
               CALWR_WQX-A0612000    2011 2011-05-04       NaN  0.000261   
                                          2011-08-03       NaN  0.000445   
                                          2011-11-02       NaN  0.000572   
                                     2012 2012-02-01       NaN  0.000488   
                                          2012-05-09       NaN  0.000427   
                                          2012-08-08       NaN  0.000421   
                                          2012-11-07       NaN  0.000621   
                                     2013 2013-02-07       NaN  0.000458   
                                          2013-05-08       NaN  0.000458   
                                          2013-08-07       NaN  0.000319   
                                          2013-11-06       NaN  0.000519   
                                     2014 2014-02-05       NaN  0.000571   
                                          2014-05-07       NaN  0.000550   
                                          2014-08-13       NaN  0.000718   
               CALWR_WQX-A610

## Padding station-level data

In [194]:
%%time
f = lambda x: x.fillna(method='pad').fillna(method='backfill')
filled = dailies.groupby(level=[0,1,2]).transform(f).reset_index()

CPU times: user 7.29 s, sys: 78.2 ms, total: 7.37 s
Wall time: 7.47 s


In [195]:
filled

CountyName     LocationIdentifier  Year  StartDate     Value  \
Pollutant                                                          Antimony   
0          Alameda County  CALWR_WQX-B9C74701355  2010 2010-03-17       NaN   
1          Alameda County  CALWR_WQX-B9C74701355  2010 2010-04-14       NaN   
2          Alameda County  CALWR_WQX-B9C74701355  2010 2010-06-23       NaN   
3          Alameda County  CALWR_WQX-B9C74701355  2010 2010-07-21       NaN   
4          Alameda County  CALWR_WQX-B9C74701355  2010 2010-08-30       NaN   
5          Alameda County  CALWR_WQX-B9C74701355  2010 2010-09-15       NaN   
6          Alameda County  CALWR_WQX-B9C74701355  2010 2010-10-13       NaN   
7          Alameda County  CALWR_WQX-B9C74701355  2010 2010-10-26       NaN   
8          Alameda County  CALWR_WQX-B9C74701355  2010 2010-11-03       NaN   
9          Alameda County  CALWR_WQX-B9C74701355  2010 2010-11-09       NaN   
10         Alameda County  CALWR_WQX-B9C74701355  2010 2010-12-07       NaN   
11         Alameda County  CALWR_WQX-B9C74701355  2010 2010-12-08       NaN   
12         Alameda County  CALWR_WQX-B9C74701355  2011 2011-01-04       NaN   
13         Alameda County  CALWR_WQX-B9C74701355  2011 2011-01-05       NaN   
14         Alameda County  CALWR_WQX-B9C74701355  2011 2011-01-19       NaN   
15         Alameda County  CALWR_WQX-B9C74701355  2011 2011-02-02       NaN   
16         Alameda County  CALWR_WQX-B9C74701355  2011 2011-02-08       NaN   
17         Alameda County  CALWR_WQX-B9C74701355  2011 2011-03-02       NaN   
18         Alameda County  CALWR_WQX-B9C74701355  2011 2011-03-08       NaN   
19         Alameda County  CALWR_WQX-B9C74701355  2011 2011-03-23       NaN   
20         Alameda County  CALWR_WQX-B9C74701355  2011 2011-04-05       NaN   
21         Alameda County  CALWR_WQX-B9C74701355  2011 2011-04-07       NaN   
22         Alameda County  CALWR_WQX-B9C74701355  2011 2011-05-03       NaN   
23         Alameda County  CALWR_WQX-B9C74701355  2011 2011-05-04       NaN   
24         Alameda County  CALWR_WQX-B9C74701355  2011 2011-06-01       NaN   
25         Alameda County  CALWR_WQX-B9C74701355  2011 2011-06-07       NaN   
26         Alameda County  CALWR_WQX-B9C74701355  2011 2011-06-15       NaN   
27         Alameda County  CALWR_WQX-B9C74701355  2011 2011-07-06       NaN   
28         Alameda County  CALWR_WQX-B9C74701355  2011 2011-07-12       NaN   
29         Alameda County  CALWR_WQX-B9C74701355  2011 2011-08-03       NaN   
...                   ...                    ...   ...        ...       ...   
25662         Yuba County     CALWR_WQX-A0052500  2010 2010-01-26       NaN   
25663         Yuba County     CALWR_WQX-A0052500  2010 2010-03-02       NaN   
25664         Yuba County     CALWR_WQX-A0052500  2010 2010-03-24       NaN   
25665         Yuba County     CALWR_WQX-A0052500  2010 2010-04-21       NaN   
25666         Yuba County     CALWR_WQX-A0052500  2010 2010-05-26       NaN   
25667         Yuba County     CALWR_WQX-A0612000  2011 2011-05-04       NaN   
25668         Yuba County     CALWR_WQX-A0612000  2011 2011-08-03       NaN   
25669         Yuba County     CALWR_WQX-A0612000  2011 2011-11-02       NaN   
25670         Yuba County     CALWR_WQX-A0612000  2012 2012-02-01       NaN   
25671         Yuba County     CALWR_WQX-A0612000  2012 2012-05-09       NaN   
25672         Yuba County     CALWR_WQX-A0612000  2012 2012-08-08       NaN   
25673         Yuba County     CALWR_WQX-A0612000  2012 2012-11-07       NaN   
25674         Yuba County     CALWR_WQX-A0612000  2013 2013-02-07       NaN   
25675         Yuba County     CALWR_WQX-A0612000  2013 2013-05-08       NaN   
25676         Yuba County     CALWR_WQX-A0612000  2013 2013-08-07       NaN   
25677         Yuba County     CALWR_WQX-A0612000  2013 2013-11-06       NaN   
25678         Yuba County     CALWR_WQX-A0612000  2014 2014-02-05       NaN   
25679         Yuba County     CALWR_WQX-A0612000  2014 2014-05-07

## Filling county means for missing station data

In [198]:
countyMeans = imputed.sort_values(['Year', 'CountyName'])
countyMeans.set_index(['Year', 'CountyName'], inplace=True)

def fillnasForCounty(county, df, defaults):
    countydf = df.loc[df.CountyName == county]
    countydefaults = defaults.ix[county]
    return countydf.fillna(countydefaults)

def fillnasForYear(year, df):
    yeardf = df.loc[df.Year == year]
    yeardefaults = countyMeans.ix[year]
    yeardf = pd.concat([
            fillnasForCounty(county, yeardf, yeardefaults)
            for county in yeardf.CountyName.unique()
        ])
    return yeardf

reallyFilled = pd.concat([
        fillnasForYear(year, filled)
        for year in filled.Year.unique()
    ])

In [199]:
reallyFilled.describe()

Year         Value                              \
Pollutant                    Antimony       Arsenic        Barium   
count      25692.000000  25692.000000  25692.000000  25692.000000   
mean        2012.174296      0.000137      0.004931      0.078691   
std            1.723450      0.000326      0.023611      0.106453   
min         2010.000000      0.000000     -0.000200      0.000000   
25%         2011.000000      0.000003      0.001650      0.030218   
50%         2012.000000      0.000072      0.002692      0.052000   
75%         2013.000000      0.000134      0.003800      0.097450   
max         2016.000000      0.015650      1.040000      4.930000   

                                                                   \
Pollutant     Beryllium       Cadmium      Chromium        Copper   
count      25692.000000  25692.000000  25692.000000  25692.000000   
mean           0.000097      0.000090      0.006060      0.004347   
std            0.000363      0.000305      0.026986      0.009771   
min            0.000000     -0.000150     -0.000170     -0.000150   
25%            0.000000      0.000010      0.000500      0.001555   
50%            0.000007      0.000031      0.001341      0.002542   
75%            0.000025      0.000083      0.003401      0.004288   
max            0.003210      0.012600      0.611017      0.620000   

                                                                   \
Pollutant      Fluoride          HAA5          Lead       Mercury   
count      25692.000000  25692.000000  25692.000000  2.569200e+04   
mean           0.379536      0.122011      0.000921  2.287923e-05   
std            1.187736      0.034499      0.003267  6.890790e-05   
min           -0.023000      0.001300     -0.000150 -2.000000e-07   
25%            0.160000      0.103955      0.000115  0.000000e+00   
50%            0.224286      0.113303      0.000354  6.733333e-07   
75%            0.382899      0.147462      0.000800  1.531750e-05   
max          150.000000      0.700000      0.140000  5.000000e-03   

                                                                   \
Pollutant       Nitrate       Nitrite          PCBs      Selenium   
count      25692.000000  25692.000000  25692.000000  25692.000000   
mean           8.398591      0.122208      0.000022      0.004352   
std           28.167796      0.447555      0.000141      0.016524   
min           -0.617500     -0.996000     -0.002806     -0.000560   
25%            0.400000      0.010000      0.000000      0.000190   
50%            2.200000      0.029583      0.000000      0.000700   
75%            8.887351      0.120000      0.000027      0.002072   
max         3035.000000     33.200000      0.000647      0.548000   

                                                     
Pollutant      Simazine         TTHMs        Xylene  
count      25692.000000  25692.000000  25692.000000  
mean           0.000149      0.042751      0.000036  
std            0.000453      0.097561      0.000097  
min            0.000000      0.000000      0.000000  
25%            0.000009      0.000030      0.000000  
50%            0.000034      0.000325      0.000006  
75%            0.000159      0.011200      0.000042  
max            0.020100      1.851000      0.001159